In [1]:
# This is file for train, prediction

import pandas as pd

dirpath = "C:/Users/rihot/Desktop/Deep_learning/DACON_used_car_price/"

train = pd.read_csv('data/preprocessed_train.csv')
test = pd.read_csv('data/preprocessed_test.csv')

In [2]:
from sklearn import model_selection

# train data for scaler fitting

Y = train[ ['target'] ].values
X = train[ ['title', 'odometer', 'isimported', 'engine', 'paint', 'year', 'brand' ] ].values
X_test = test[ ['title', 'odometer', 'isimported', 'engine', 'paint', 'year', 'brand' ] ].values
# X = train[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values
# X_test = test[ ['title', 'odometer', 'location', 'isimported', 'engine', 'transmission', 'fuel', 'paint', 'year', 'brand' ] ].values

In [3]:
import numpy as np

X1 = train[['odometer']].values
X2 = train[['title','isimported','engine','paint','year','brand']].values

X3 = test[['odometer']].values
X4 = test[['title','isimported','engine','paint','year','brand']].values

Y = np.log(Y)

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scalerX1 = MinMaxScaler()
scalerX1.fit(X1)
X1 = scalerX1.transform(X1)
X3 = scalerX1.transform(X3)

scalerX2 = StandardScaler()
scalerX2.fit(X2)
X2 = scalerX2.transform(X2)
X4 = scalerX2.transform(X4)

X = np.concatenate([X1,X2], axis=1)
X_test = np.concatenate([X3, X4], axis=1)

scalerY = MinMaxScaler()
scalerY.fit(Y)
Y = scalerY.transform(Y)

In [5]:
x_train, x_val, y_train, y_val = \
    model_selection.train_test_split(X, Y, test_size=0.3, random_state=41)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((710, 7), (305, 7), (710, 1), (305, 1))

In [6]:
from pycaret import regression

model = regression.setup(data=train, target='target',train_size=0.7,session_id=0,silent=True)

,Description,Value
0,session_id,0
1,Target,target
2,Original Data,"(1015, 11)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(710, 38)"


In [7]:
top_3_models = regression.compare_models(sort='MSE', n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,2998808.8289,75950386794725.6719,7562803.0964,0.6258,0.5428,0.4476,0.8840
gbr,Gradient Boosting Regressor,3367147.1932,83225270948606.5469,8197183.7426,0.5353,0.5597,0.5396,0.0520
rf,Random Forest Regressor,3437288.8441,86758386100494.2812,8315563.9961,0.5335,0.5213,0.4928,0.2080
lightgbm,Light Gradient Boosting Machine,3645904.3172,89579974505431.5625,8530687.6092,0.5143,0.6515,0.5437,0.0410
et,Extra Trees Regressor,3461602.5775,91511148612644.7500,8739773.7582,0.4780,0.5220,0.4882,0.1830
xgboost,Extreme Gradient Boosting,3257962.7301,94958873191807.8281,8760960.6525,0.4919,0.5439,0.4659,0.1500
ridge,Ridge Regression,5291788.2197,110492228934623.4688,9760337.4280,0.3316,0.9066,1.1957,0.0060
lr,Linear Regression,5395634.8901,111149343370582.4688,9792836.6546,0.3269,0.9172,1.2310,1.3750
lasso,Lasso Regression,5341211.0254,111154922462693.2500,9794632.0376,0.3246,0.9312,1.2057,0.0100
llar,Lasso Least Angle Regression,5343691.4310,111184035217902.5000,9796278.0396,0.3243,0.9314,1.2069,0.0090


In [8]:
blended = regression.blend_models(estimator_list=top_3_models, fold=10, optimize='MSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2261046.2670,18823192171817.6562,4338570.2912,0.8302,0.4020,0.3702
1,4508685.9363,325626732255125.8125,18045130.4305,0.1848,0.5977,0.3966
2,3154894.3184,35693971248910.6172,5974443.1748,0.7079,0.6176,0.7345
3,3360201.3573,122777923515790.4062,11080520.0020,0.4695,0.4953,0.4692
4,2154382.7714,10530734240426.4004,3245109.2802,0.8206,0.4169,0.3907
5,3194956.9813,37935031306914.1406,6159142.0918,0.5376,0.6179,0.7179
6,4434595.2319,122386006013981.2656,11062820.8886,0.5604,0.5505,0.5318
7,2955293.2138,37932975561089.1641,6158975.2038,0.4897,0.4933,0.3322
8,2018911.0601,22293009586060.6094,4721547.3720,0.6600,0.3925,0.3236


In [9]:
import numpy as np
from sklearn.metrics import mean_absolute_error

from itertools import combinations
from vecstack import StackingTransformer
from tqdm import tqdm

def NMAE(true, pred):
    mae = mean_absolute_error(true,pred)
    score = mae / np.mean(np.abs(true))
    return score

In [10]:
print(X.shape, X_test.shape, Y.shape)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

(1015, 7) (436, 7) (1015, 1)
(710, 7) (305, 7) (710, 1) (305, 1)


In [11]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

models = [
    ('LGBMRegressor',LGBMRegressor(n_estimators=500,max_depth=12, learning_rate=0.02,subsample=0.8,random_state=0)),
    ('XGBRegressor',XGBRegressor(n_estimators=900,max_depth=16,learning_rate=0.02,random_state=0,subsample=1.0,verbosity=0)),
    ('CatBoostRegressor',CatBoostRegressor(n_estimators=800,max_depth=10,learning_rate=0.02,subsample=0.8,random_state=0,verbose=0)),
    ('GradientBoostingRegressor',GradientBoostingRegressor(n_estimators=700,max_depth=9,learning_rate=0.05,subsample=0.8,random_state=0)),
    ('ExtraTreesRegressor',ExtraTreesRegressor(n_estimators=600,max_depth=9,random_state=0,criterion='mse')),
    ('RandomForestRegressor',RandomForestRegressor(n_estimators=500,max_depth=12,random_state=0,criterion='mse'))]

In [12]:
for name, model in models:
    model.fit(x_train, y_train)
    print(f'{name}: ', NMAE(y_val,model.predict(x_val)))

LGBMRegressor:  0.12792283684879552
XGBRegressor:  0.14027052129726658
CatBoostRegressor:  0.11657556170887975
GradientBoostingRegressor:  0.1292094702519579
ExtraTreesRegressor:  0.12902322240833594
RandomForestRegressor:  0.1344326565211613


In [13]:
pred_list = list()
for name, model in models:
    pred_list.append([model.predict(x_val),name])

pred_comb = list()
for i in range(2,len(pred_list)+1):
    pred_comb += list(combinations(pred_list, i))

In [14]:
nmae_list = list()

for pred_info in pred_comb:
    preds, names = list(), list()

    for pred, name in pred_info:
        preds.append(pred)
        names.append(name)
    nmae_list.append([NMAE(y_val,sum(preds)/len(preds)),names])

In [15]:
sorted(nmae_list)[0]

[0.11885160274910317, ['CatBoostRegressor', 'ExtraTreesRegressor']]

In [16]:
estimator = [('XGBRegressor',XGBRegressor(n_estimators=900,max_depth=16,learning_rate=0.02,random_state=0,subsample=1.0,verbosity=0)),
('CatBoostRegressor',CatBoostRegressor(n_estimators=800,max_depth=10,learning_rate=0.02,subsample=0.8,random_state=0,verbose=0))]

In [17]:
stack = StackingTransformer(estimator,
                            regression = True,
                            metric = NMAE,
                            n_folds = 10, stratified = True, shuffle = True,
                            random_state = 0, verbose = 0)

In [18]:
S_train = stack.fit_transform(x_train, y_train)
S_test = stack.transform(x_val)

S_model = LGBMRegressor(n_estimators=500,max_depth=12,random_state=0)
S_model.fit(S_train, y_train)
NMAE(y_val, S_model.predict(S_test))

0.14496489677550056

In [19]:
models = [
    ('XGBRegressor',XGBRegressor(n_estimators=900,max_depth=16,learning_rate=0.02,random_state=0,subsample=1.0,verbosity=0)),
    ('CatBoostRegressor',CatBoostRegressor(n_estimators=800,max_depth=10,learning_rate=0.02,subsample=0.8,random_state=0,verbose=0))
]

In [20]:
pred = None
for name, model in models:
    model.fit(x_train, y_train)
    if pred is None:
        pred = model.predict(X_test)
    else:
        pred += model.predict(X_test)

pred = pred/len(models)
pred = scalerY.inverse_transform([pred])
pred = np.exp(pred)

In [21]:
pred = pred.reshape(-1,1)

In [22]:
pred.shape

(436, 1)

In [23]:
sample = pd.read_csv('sample_submission.csv')
sample['target'] = pred

sample.to_csv( dirpath + "submit.csv", index=False)